In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pickle
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
df = pd.read_csv("/home/growlt199/Downloads/manufacturing_project/data/flight_processed_data.csv",index_col=0)

In [3]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,0,Night,Mumbai,0,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,0,Morning,Mumbai,0,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,0,Afternoon,Mumbai,0,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,0,Morning,Mumbai,0,2.33,1,5955


In [4]:
#done label encoding in flight column
label_encoder = LabelEncoder()
df['flight'] = label_encoder.fit_transform(df['flight'])

In [5]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,1408,Delhi,Evening,0,Night,Mumbai,0,2.17,1,5953
1,SpiceJet,1387,Delhi,Early_Morning,0,Morning,Mumbai,0,2.33,1,5953
2,AirAsia,1213,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,2.17,1,5956
3,Vistara,1559,Delhi,Morning,0,Afternoon,Mumbai,0,2.25,1,5955
4,Vistara,1549,Delhi,Morning,0,Morning,Mumbai,0,2.33,1,5955


In [6]:
categorical_columns = ['airline', 'departure_time','arrival_time','source_city','destination_city']

# Apply label encoding to each categorical column
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

In [7]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,4,1408,2,2,0,5,5,0,2.17,1,5953
1,4,1387,2,1,0,4,5,0,2.33,1,5953
2,0,1213,2,1,0,1,5,0,2.17,1,5956
3,5,1559,2,4,0,0,5,0,2.25,1,5955
4,5,1549,2,4,0,4,5,0,2.33,1,5955


In [8]:
#divide into x and y
X = df.drop('price',axis=1)
y = df['price']

In [9]:
X.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left
0,4,1408,2,2,0,5,5,0,2.17,1
1,4,1387,2,1,0,4,5,0,2.33,1
2,0,1213,2,1,0,1,5,0,2.17,1
3,5,1559,2,4,0,0,5,0,2.25,1
4,5,1549,2,4,0,4,5,0,2.33,1


In [10]:
y.head()

0    5953
1    5953
2    5956
3    5955
4    5955
Name: price, dtype: int64

In [11]:
#use sklearn train test spilt for spilt our data in parts like training data and testing data 
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state =101, shuffle=True)

In [12]:
X_train.shape

(239063, 10)

In [13]:
X_test.shape

(59766, 10)

In [14]:
#scaling in x train and y train for model building and testing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
print(X_train_scaled[:10])

[[ 1.0353104   0.86278963  0.81262437  1.47248555  0.1932792  -1.76404208
  -0.33642355 -0.67083373  0.74897036  0.14617432]
 [-1.14827428 -0.45210827  0.81262437 -0.8072832   0.1932792  -0.61640925
   1.3831308  -0.67083373 -0.35186278 -0.14920206]
 [-1.14827428 -0.72868216  1.38403905 -0.8072832  -2.32201227 -1.19022567
  -0.90960833  1.49068235 -1.43020124  1.32767982]
 [-1.14827428 -0.5294552  -1.47303435  1.47248555 -2.32201227  1.10503999
  -0.33642355 -0.67083373 -1.30085686 -1.69992803]
 [ 1.0353104   0.87450886 -1.47303435 -0.23734101  0.1932792  -1.19022567
   1.3831308  -0.67083373  0.12755498 -1.10917528]
 [-0.60237811 -0.05599821  0.24120969  0.90254336  0.1932792  -0.61640925
  -0.33642355 -0.67083373 -0.6569468   0.07233023]
 [ 1.0353104   1.0479535   1.38403905 -0.8072832   0.1932792  -0.61640925
   0.23676123  1.49068235 -0.37576337 -1.25686346]
 [ 0.48941423  0.51824419  0.81262437 -0.23734101  0.1932792   0.53122357
  -0.33642355 -0.67083373  0.7025751   0.81077116]


In [16]:
print(X_test_scaled[:10])

[[ 1.0353104   1.05498504 -0.33020499 -0.23734101  0.1932792  -1.19022567
  -1.48279311  1.49068235  0.40873841  1.47536801]
 [-1.14827428 -0.56695675  0.81262437  1.47248555  0.1932792   0.53122357
   0.23676123 -0.67083373  0.45653959 -0.14920206]
 [-1.14827428 -0.75680832  0.24120969  0.90254336  0.1932792  -0.04259284
  -1.48279311  1.49068235  0.09240705  0.58923888]
 [-0.60237811 -0.17084669 -0.33020499  0.90254336  0.1932792  -0.61640925
  -1.48279311 -0.67083373 -0.91422964 -1.33070756]
 [-0.05648194 -1.7435677  -0.90161967 -0.8072832  -2.32201227  0.53122357
   1.3831308  -0.67083373 -1.43020124  0.81077116]
 [ 1.0353104   0.89560348 -0.33020499  0.90254336 -2.32201227  0.53122357
  -1.48279311 -0.67083373 -1.32475745  1.17999163]
 [-0.05648194 -1.98967158 -1.47303435 -0.8072832   0.1932792  -0.61640925
  -0.33642355 -0.67083373 -0.45730656 -1.25686346]
 [ 1.0353104   0.92607348 -0.90161967 -0.8072832   0.1932792   0.53122357
   0.80994601  1.49068235  2.0381964  -0.29689024]


# building basic neural network

In [17]:
model = Sequential()

# Add input layer and the first hidden layer
model.add(Dense(64, activation='relu', input_dim=10))

# Add additional hidden layers
model.add(Dense(16, activation='relu'))
model.add(Dense(4, activation='relu'))
# Add output layer
model.add(Dense(1))

In [18]:
import time
start_time = time.time()

model.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])

#print model summary
print(model.summary())

# fiting data in neural network
model.fit(X_train, y_train, epochs=500, batch_size=32)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               1408      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dense_3 (Dense)             (None, 16)                528       
                                                                 
 dense_4 (Dense)             (None, 8)                 136       
                                                                 
 dense_5 (Dense)             (None, 4)                 36        
                                                                 
 dense_6 (Dense)             (None, 1)                 5

7471/7471 [==============================] - 6s 747us/step - loss: 15970638.0000 - mae: 2402.2678
Epoch 136/500
7471/7471 [==============================] - 6s 764us/step - loss: 15955791.0000 - mae: 2395.8342
Epoch 137/500
7471/7471 [==============================] - 6s 780us/step - loss: 15960516.0000 - mae: 2399.8206
Epoch 138/500
7471/7471 [==============================] - 6s 753us/step - loss: 15898541.0000 - mae: 2397.1665
Epoch 139/500
7471/7471 [==============================] - 6s 747us/step - loss: 16010833.0000 - mae: 2397.3894
Epoch 140/500
7471/7471 [==============================] - 6s 780us/step - loss: 15794800.0000 - mae: 2388.9094
Epoch 141/500
7471/7471 [==============================] - 6s 757us/step - loss: 15858438.0000 - mae: 2390.9539
Epoch 142/500
7471/7471 [==============================] - 6s 764us/step - loss: 15869602.0000 - mae: 2392.9482
Epoch 143/500
7471/7471 [==============================] - 6s 779us/step - loss: 15747473.0000 - mae: 2379.4294
Epoch 

7471/7471 [==============================] - 6s 802us/step - loss: 13630486.0000 - mae: 2202.8337
Epoch 282/500
7471/7471 [==============================] - 5s 722us/step - loss: 13603230.0000 - mae: 2203.1833
Epoch 283/500
7471/7471 [==============================] - 5s 726us/step - loss: 13671023.0000 - mae: 2204.9031
Epoch 284/500
7471/7471 [==============================] - 6s 748us/step - loss: 13553205.0000 - mae: 2196.5969
Epoch 285/500
7471/7471 [==============================] - 6s 737us/step - loss: 13590926.0000 - mae: 2201.4458
Epoch 286/500
7471/7471 [==============================] - 5s 725us/step - loss: 13563972.0000 - mae: 2196.9624
Epoch 287/500
7471/7471 [==============================] - 5s 735us/step - loss: 13502440.0000 - mae: 2193.2183
Epoch 288/500
7471/7471 [==============================] - 6s 740us/step - loss: 13548937.0000 - mae: 2197.3083
Epoch 289/500
7471/7471 [==============================] - 5s 719us/step - loss: 13543445.0000 - mae: 2192.9783
Epoch 

7471/7471 [==============================] - 6s 761us/step - loss: 12499403.0000 - mae: 2108.9272
Epoch 428/500
7471/7471 [==============================] - 6s 757us/step - loss: 12497107.0000 - mae: 2107.5396
Epoch 429/500
7471/7471 [==============================] - 6s 747us/step - loss: 12598626.0000 - mae: 2113.9028
Epoch 430/500
7471/7471 [==============================] - 6s 762us/step - loss: 12486755.0000 - mae: 2104.1255
Epoch 431/500
7471/7471 [==============================] - 6s 764us/step - loss: 12516518.0000 - mae: 2111.6821
Epoch 432/500
7471/7471 [==============================] - 5s 725us/step - loss: 12490292.0000 - mae: 2110.0928
Epoch 433/500
7471/7471 [==============================] - 6s 765us/step - loss: 12514472.0000 - mae: 2107.3176
Epoch 434/500
7471/7471 [==============================] - 6s 754us/step - loss: 12446894.0000 - mae: 2101.0603
Epoch 435/500
7471/7471 [==============================] - 6s 761us/step - loss: 12479502.0000 - mae: 2105.2229
Epoch 

In [19]:
y_pred = model.predict(X_test)

1868/1868 [==============================] - 1s 424us/step


In [20]:
# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)

print("MAE:", mae)
print("MSE:", mse)

MAE: 2051.6328151398934
MSE: 11968871.38105717


In [ ]:
with open('filename', 'wb') as file:
        pickle.dump(model, file)

# second model

In [21]:
model1 = Sequential()

# Add input layer and the first hidden layer
model1.add(Dense(512, activation='relu', input_dim=10))

# Add additional hidden layers
model1.add(Dense(256, activation='relu'))
model1.add(Dense(128, activation='relu'))
model1.add(Dense(64, activation='relu'))
model1.add(Dense(32, activation='relu'))
model1.add(Dense(16, activation='relu'))
model1.add(Dense(8, activation='relu'))
model1.add(Dense(4, activation='relu'))
# Add output layer
model1.add(Dense(1))

In [ ]:
import time
start_time = time.time()

model1.compile(optimizer='adam', loss='mean_squared_error',metrics=['mae'])

#print model summary
print(model1.summary())

# fiting data in neural network
model1.fit(X_train, y_train, epochs=500, batch_size=32)

end_time = time.time()
execution_time = end_time - start_time
print("Execution time:", execution_time, "seconds")

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 512)               5632      
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 dense_9 (Dense)             (None, 128)               32896     
                                                                 
 dense_10 (Dense)            (None, 64)                8256      
                                                                 
 dense_11 (Dense)            (None, 32)                2080      
                                                                 
 dense_12 (Dense)            (None, 16)                528       
                                                                 
 dense_13 (Dense)            (None, 8)                

7471/7471 [==============================] - 14s 2ms/step - loss: 12722251.0000 - mae: 2061.4504
Epoch 135/500
7471/7471 [==============================] - 14s 2ms/step - loss: 12571009.0000 - mae: 2058.4326
Epoch 136/500
7471/7471 [==============================] - 14s 2ms/step - loss: 12659171.0000 - mae: 2060.9199
Epoch 137/500
7471/7471 [==============================] - 14s 2ms/step - loss: 12492629.0000 - mae: 2044.8551
Epoch 138/500
7471/7471 [==============================] - 15s 2ms/step - loss: 12670833.0000 - mae: 2058.1157
Epoch 139/500
7471/7471 [==============================] - 14s 2ms/step - loss: 12604760.0000 - mae: 2056.6099
Epoch 140/500
7471/7471 [==============================] - 14s 2ms/step - loss: 12452569.0000 - mae: 2045.8763
Epoch 141/500
7471/7471 [==============================] - 15s 2ms/step - loss: 12465990.0000 - mae: 2041.7529
Epoch 142/500
4385/7471 [================>.............] - ETA: 6s - loss: 12487223.0000 - mae: 2043.2753

In [ ]:
y_pred1 = model1.predict(X_test)

In [ ]:
# Calculate Mean Absolute Error (MAE)
mae1 = mean_absolute_error(y_test, y_pred1)

# Calculate Mean Squared Error (MSE)
mse1 = mean_squared_error(y_test, y_pred1)

print("MAE:", mae1)
print("MSE:", mse1)